In [1]:
class DatasetBase:
    def __init__(self, bucket_name_input, path_input, delimiter_input, extension):
        self.bucket_name_input = bucket_name_input
        self.path_input = path_input
        self.delimiter_input = delimiter_input
        self.extension = extension
        
    def setVariables(self):
        self.complete_path_input = '/mnt/' + self.bucket_name_input + self.path_input
        self.filename_input = self.path_input.split('/')[-1].split('.')[0]
        self.extension_input = self.path_input.split('/')[-1].split('.')[1]
        
        if self.path_input.split('/')[-1].split('.')[-1] == self.extension_input:
            self.compression_input = ''
        else:
            self.compression_input = self.path_input.split('/')[-1].split('.')[2]
            
        self.bucket_name_output = dbutils.secrets.get(scope = "aws", key = "aws-bucket-name")
        self.complete_path_output = '/mnt/' + self.bucket_name_output + '/raw/' + self.filename_input + '/' 
     
    @staticmethod
    def loadDatabase(bucket_name, complete_path, extension='parquet', delimiter=','):
        DatasetBase.createBucket(bucket_name)
        
        if extension == 'csv':
            df = spark.read.options(header=True,delimiter=delimiter).csv(complete_path)
        elif extension == 'json':
            df = spark.read.json(complete_path)
        elif extension == 'parquet':
            df = spark.read.parquet(complete_path)
        else:
            raise 'ERROR: Formato não compatível'
            
        return df
    
    @staticmethod
    def writeDataset(df, bucket_name_output, complete_path_output):
        DatasetBase.createBucket(bucket_name_output)
        df.write.format('parquet').mode("overwrite").save(complete_path_output)
        return df
      
    @staticmethod
    def createBucket(bucket_name):
        access_key = dbutils.secrets.get(scope = "aws", key = "aws-access-key")
        secret_key = dbutils.secrets.get(scope = "aws", key = "aws-secret-key").replace("/", "%2F")
        encription = "sse-s3"
        
        if not any(mount.mountPoint == f'/mnt/{bucket_name}' for mount in dbutils.fs.mounts()):
            dbutils.fs.mount(f's3a://{access_key}:{secret_key}@{bucket_name}', f'/mnt/{bucket_name}', encription)            
        return dbutils.fs.mounts()
    
    @staticmethod
    def deleteBucket(bucket_name):
        if any(mount.mountPoint == f'/mnt/{bucket_name}' for mount in dbutils.fs.mounts()):
            dbutils.fs.unmount(f'/mnt/{bucket_name}')        
        return dbutils.fs.mounts()

Out[59]: True